In [9]:
from datasets import load_dataset
import random

In [2]:
dataset = load_dataset("akoksal/LongForm")

Using custom data configuration akoksal--LongForm-e912efe86f9a3c91


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2066.84it/s]


Dataset parquet downloaded and prepared to /data/tianle/.cache/huggingface/datasets/akoksal___parquet/akoksal--LongForm-e912efe86f9a3c91/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 435.29it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'source', 'subset'],
        num_rows: 23652
    })
    test: Dataset({
        features: ['input', 'output', 'source', 'subset'],
        num_rows: 2045
    })
    validation: Dataset({
        features: ['input', 'output', 'source', 'subset'],
        num_rows: 2042
    })
})

In [19]:
uniform_len = [10, 20, 30, 40, 50, 60, 70, 80]
non_uniform_len = [16, 22, 28, 34, 56, 62, 68, 74]
ranking_dataset = []
num_ans = 8
for data in dataset["train"]:
    answer = data["output"].split(" ")
    if len(answer) < 100:
        continue
    ranking_data = {}
    uniform = (random.random() > 0.5)
    ranking_data["uniform"] = uniform
    ranking_data["question"] = data["input"] + f" Write the answer in an open-ended way." if uniform else data["input"] + f" Write either a short answer or a long answer."
    ranking_data["answers"] = []
    if uniform:
        for i in range(num_ans):
            ranking_data["answers"].append(" ".join(answer[:uniform_len[i]]))
    else:
        for i in range(num_ans):
            ranking_data["answers"].append(" ".join(answer[:non_uniform_len[i]]))
    ranking_data["pairs"] = []
    for i in range(num_ans):
        for j in range(i + 1, num_ans):
            ranking_data["pairs"].append([ranking_data["answers"][j], ranking_data["answers"][i]])
    ranking_dataset.append(ranking_data)

In [20]:
len(ranking_dataset)

15025

In [12]:
import torch

In [21]:
torch.save(ranking_dataset, "ranking_dataset_longform.pt")